# Bee Foraging ABM
Based on Sam Woodman's Netlogo thesis model. Uses the agentpy ABM library.

Agenda Link: https://docs.google.com/document/d/14XbNdPlWMOrtLd1-0m3qUxlsR7LnDEyibB5Zos93G6w/

[//]: <> (Comment: markdown syntax guie https://www.markdownguide.org/basic-syntax#code-blocks)

### Current state of model:
Attribute                  | Status          | .
:-------------:            | :-------------: | :-------------:
**Agents have positions?** | Yes             | ✅
**Bees can move?**         | Yes             | ✅
**Model animated?**        | Yes             | ✅
**Memory?**                | No              | ❌
**Communication?**         | No              | ❌


### Simplifications:
* **Resources:** 
    * Randomly distributed
    * All patches are the same size
    * No difference in quality / quantity
    * Cannot be depleted / are not ephemeral
    
* **Bee movement:** 
    * Random angle from -45º to 45º relative to current orientation. 
    * Distance pulled from uniform distribution from 0 to dist_per_tick parameter 
        * (dist_per_tick is used for distance bees move per tick when returning to hive)
    * Foraging time at resource is constant
    
* **Communication:** Off

* **Memory:** Off

### Other assumptions:
* Bees can overlap each other
* Bees more slower when exploring than they do when returning
* Bees return directly to hive in a straight line
* Bees only forage at one resource per trip
* Bees are not influenced by the presence of other bees on nearby resources
* The landscape is square, and the hive is in the center
* The returning bees don't spend any time unloading pollen/nectar (they can forage again immediately)
* Currently the model doesn't destinguish between pollen and nectar foraging, but I expect in future versions it will focus specifically on nectar (with the assumption all our bees are pollen foragers)

### Known issues:
* **General**
    * I think there might be some weird stuff with the order you need to run the chunks- not totally sure about that though

* **Animations**
    * One has the bees the same color as the plants (confusing!)
    * The other crashes when you hit the play button
    
* **Beehavior**
    * In some cases, the angle for a bee's return to hive should be negative (all values given now are positive)
        * Can be easily repared with an if statement for cases, looking to see if there's a more elegant solution


██████████████████████████████████████████

### The below cell allows us to change the background of color other cells! 

This works in Jupyter, but I'm not sure about other ipython viewers.
Changing the color might sound superfluous, but I guarantee it's super, super helpful for clarifying which class you're looking at.

#### Here's how to use it:
1. Run the below cell
2. For each cell:
    1. Un-comment the first line (it should look like `#%%bgc #FFB7B2`)
    2. Run the cell
    3. Re-comment the first line
3. You're done!

In [19]:
# https://www.w3schools.com/colors/colors_palettes.asp
# https://stackoverflow.com/questions/49429585/how-to-change-the-background-color-of-a-single-cell-in-a-jupyter-notebook-jupy#comment104601184_50824920
from IPython.core.magic import register_cell_magic
from IPython.display import HTML, display
my_palette = ['#edc6a3','#f4ce88','#f0e9ee','#cbcfd9','#c9dac7','#d1d3be','#efe1d5','#e6b4a7','#c0b6a2','#cdb090']
my_palette_names = {'getter':'#e6b4a7', 'init':'#efe1d5'} 
@register_cell_magic
def bgc(color, cell=None):
    if str.isdigit(color) and len(color) == 1:
        color = my_palette[int(color)]
    else:
        color = my_palette_names.get(color, color)
    script = ("var cell = this.closest('.code_cell');"
    "var editor = cell.querySelector('.input_area');"
    "editor.style.background='{}';"
    "this.parentNode.removeChild(this)").format(color) 
    display(HTML('<img src onerror="{}">'.format(script)))

██████████████████████████████████████████

This model is built using agentpy in an object-oriented way. Each kind of agent (here bees, resources, and the hive) has its own class. The model itself is also a class, called `ForagingModel`. 

Agents are created as instances of their respective classes. They are created with certain variable values (defined by their class), such as their position and state. They can be grouped together by these variables in groups called AgentLists.

In [20]:
#%%bgc #FFB7B2

#Model
import agentpy as ap
print("Agentpy Version: ", ap.__version__)
import numpy as np

#Visualization
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors #for calculating R value
import seaborn as sns
import IPython
import ipysimulate as ips
print("ips version: ", ips.__version__)
from ipywidgets import AppLayout
import itertools

def dist_btwn(coordsA, coordsB):
        """
        Given two (2D) points, find the Cartesian distance between them
        """
        x1 = coordsA[0]
        x2 = coordsB[0]
        y1 = coordsA[1]
        y2 = coordsB[1]
        return np.sqrt((x1-x2)**2+(y1-y2)**2)

Agentpy Version:  0.1.2
ips version:  0.2.0


██████████████████████████████████████████

### Bee Class

In this version of the model, bees have four states:
* Inactive (hanging out at the hive)
* Searching (looking randomly for resources)
* Foraging (found a resource, extracting nectar)
* Returning (going back to the hive)

#### State Changes

* **Inactive** bees become **searching** bees with probability `p_random_forage`. 

* **Searching** bees become **foraging** bees if they discover one through their search (they've "found" a plant if they're within its radius from its center).

* **Searching** bees become **returning** bees if they have been searching for `give_up_threshold` time ticks. 

* **Foraging** bees become **returning** bees after they have been foraging at their found resource for `ticks_on_resource` time ticks.

* **Returning** bees become **inactive** once they reach the hive.

#### What they do in each state

* **Inactive** bees wait patiently at the hive. When they've reached the hive, right before their state is changed to **inactive**, their orientation is reset to a random angle from 0º to 360º (uniform distribution).

* **Searching** bees move at each time step based on an angle and a direction.
    * The angle is drawn from a uniform distribution from -45º to 45º, relative to the bee's current orientation.
    * The direction is drawn from a uniform distribution from 0 to `dist_per_tick`.

* **Foraging** bees stay in the same spot- currently, they are restricted to visiting one resource per trip. They stay there for `ticks_on_resource` time ticks.

* **Returning** bees make a bee-line towards the hive, moving at a constant rate of `dist_per_tick`.



In [121]:
#%%bgc #FFDAC1

class Bee(ap.Agent):
    """ 
    An individual bee from the colony that forages
    """
    
    def setup(self):  
        """ 
        Initialize variables at agent creation 
        """
        
        self.modinstance = model47     # Connect the bee to an instance of the model
        self.kind = "bee"        # Used to color-code animation
        
        self.orientation = model47.nprandom.uniform(low = 0.0, high = 360.0) 
        # Choose random starting orientation
        # Orientation is the direction the bee is facing [degrees] 
        # East (to the right) is 0º
        
        self.state = "inactive"  # All bees start at the hive
        
        # Possible states (strings)
        #   "inactive":   At the hive                  0
        #   "searching":  Random foraging              1
        #   "foraging":   Found a resource             2
        #   "returning":  En route back to the hive    3
        
        self.time_searching = 0  # How long bee was last "searching" for [UNIT: ticks]
        self.time_foraging  = 0  # How long bee was last "foraging"  for [UNIT: ticks]
        self.num_trips = 0       # How many times the bee has left the hive
        
    #===========================================================#

    def setup_pos(self, space):
        """
        Construct (continuous) environment
        """
        self.space = space 
        self.neighbors = space.neighbors
        self.pos = space.positions[self]
    
    #===========================================================#
        
    def move_radially(self, agent, move_dist, move_angle):
        """
        Move according to a specified distance and angle.
        move_angle is relative to the direction the bee is currently pointed in
        """
        global_angle = agent.orientation + move_angle 
        # Global angle is angle CCW to the horizontal
        # [Double check this is correct]
        
        # Bee's current coords
        current_x = agent.pos[0] 
        current_y = agent.pos[1]
        # Bee's new coords
        new_x = current_x + move_dist * np.cos(global_angle)
        new_y = current_y + move_dist * np.sin(global_angle)
        
        # Move! That! Bee!
        self.pos = [new_x,new_y]
        self.modinstance.beespace.move_to(self, self.pos)
    
    #===========================================================#
    
    def find_dist_line_seg (self, start, end, plant):
        """
        Find the distance from the center of a plant to the trajectory btwn two points (line seg)
        Used to tell if a bee will discover a resource between timesteps
        Returns a distance
        Start = initial [x,y] of bee
        End = final [x,y] of bee
        Plant = [x,y] of the center of a resource
        """
        # S = Start, E = End, P = Plant
        
        # Create vectors
        SE = [x2 - x1 for (x1, x2) in zip(start, end)]
        SP = [x2 - x1 for (x1, x2) in zip(start, plant)]
        EP = [x2 - x1 for (x1, x2) in zip(end, plant)]
        
        # Calculate dot products
        SE_EP = SE[0] * EP[0] + SE[1] * EP[1]
        SE_SP = SE[0] * SP[0] + SE[1] * SP[1]
        
        # Cases
        
        if SE_EP > 0:
            # Distance is plant-to-end
            min_dist = dist_btwn(plant, end)
            
        elif SE_SP < 0:
            # Distance is plant-to-start
            min_dist = dist_btwn(plant, start)
            
        else:
            # Distance is perp dist btwn plant and line
            x1 = SE[0]
            y1 = SE[1]
            x2 = SP[0]
            y2 = SP[1]
            mod = np.sqrt(x1 * x1 + y1 * y1)
            min_dist = abs(x1 * y2 - y1 * x2) / mod
        
        return(min_dist)
    
    #===========================================================#
    
    def new_resource_finder (self, current_pos, new_pos, midpoint, dist, angle, r_max):
        """
        Find the resources between a bee's past and future positions
        It's called when a bee is halfway between those two spots
        r_max is the maximum radius of a plant (here it's just constant)
        """
        # Select all the resources within a nearby radius
        search_radius = dist/2 + r_max
        #nearby_agents = self.modinstance.space.select(midpoint, search_radius).to_list()
        nearby_resources = self.modinstance.plantspace.select(midpoint, search_radius).to_list()
        #nearby_resources = nearby_agents.select(self.nearby_agents.state == "resource")
        #nearby_agents(nearby_agents.state == "resource")
        
        # Check if the bee would have discovered any of them on this short trajectory
        
        resource_found = False
        
        for r in nearby_resources:
            
            min_dist = self.find_dist_line_seg (current_pos, new_pos, r.pos)
            
            # If dist to plant less than plant radius
            # and plant isn't depleted, 
            # then bee found the plant and it's foraging time
            
            if min_dist <= r.radius and r.trips != 0:
                
                resource_found = True
                # If it'll discover a plant, move to near that plant's center
                
                x_jitter = model47.nprandom.uniform(low = -0.25, high = 0.25) 
                y_jitter = model47.nprandom.uniform(low = -0.25, high = 0.25) 
                jitter_pos = [r.pos[0]+x_jitter, r.pos[1]+y_jitter]
                self.modinstance.beespace.move_to(self, jitter_pos)
                
                # Deplete resource
                r.trips = r.trips - 1
                
                if r.trips ==0:
                    r.state = "depleted"
                
                self.time_foraging = 0
                self.state = "foraging"
        
        # If after checking all nearby resources, still haven't found any:
        if resource_found == False:
            #self.modinstance.beespace.move_to(self, new_pos)
            self.move_radially(self, dist, angle)

            self.state = "searching"
        
    
    #=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=# 
    ##=#=#=#=#=#=#=#=#=#=#=#  STATES  #=#=#=#=#=#=#=#=#=#=#=#=#=#
    #=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=#=# 
    
    def be_inactive(self):
        """ 
        What the inactive bees do each timestep 
        """
        rg = self.model.random  # Set up random number generator, from 0 to 1
        self.time_searching = 0    # Reset time searching counter


        if self.p.p_random_forage > rg.random():
            self.num_trips = 1 + self.num_trips
            self.state = "searching"
            
        else:
            self.state = "inactive"
            #line segment 

     #===========================================================#
            
    def search_new (self):
        # Randomly choose direction and distance
        self.time_searching += 1
        rg = self.model.random  # Set up random number generator, from 0 to 1

        
        if self.time_searching >= self.p.give_up_threshold:  # If it's been out for a long time, go home
            self.state = "returning"
            #self.go_home()
            
        elif  self.p.p_abandon_search > rg.random():
            self.state = "returning"
        
        else:
        
            move_dist = model47.nprandom.uniform(low = self.p.dist_per_tick[0], high = self.p.dist_per_tick[1])
            move_angle = model47.nprandom.uniform(low = -45, high = 45) # relative to current orientation
        
            midpoint_x = self.pos[0] + (move_dist/2) * np.cos(self.orientation + move_angle)
            midpoint_y = self.pos[1] + (move_dist/2) * np.sin(self.orientation + move_angle)
        
            new_x = self.pos[0] + (move_dist) * np.cos(self.orientation + move_angle)
            new_y = self.pos[1] + (move_dist) * np.sin(self.orientation + move_angle)

            midpoint = [midpoint_x, midpoint_y]
            new_pos = [new_x,new_y]
        
            # Check if bee will find new resources and update position / state accordingly
            self.new_resource_finder(self.pos, new_pos, midpoint, move_dist, move_angle, self.p.radius)
        

    #===========================================================#
            
    def forage(self):
        """ 
        What the foraging bees do each timestep 
        """
        
        self.time_foraging += 1         # Add 1 tick to the "time spent foraging" counter
        
        # ticks_on_resource is a parameter we set.
        # It represents how long a bee spends on a resource before going home
        
        if self.time_foraging >= self.p.ticks_on_resource:
            self.state = "returning"
        else:
            self.state = "foraging"
    
    #===========================================================#
    
    def go_home(self):
        """ 
        What the returning bees do each timestep 
        """        
        
        # Bee's current position
        current_x = self.pos[0]
        current_y = self.pos[1]

        # If the bee is close enough to the hive (i.e. it could get there
        # within one timestep), it goes there.
        
        if dist_btwn(self.pos, [self.p.land_size/2, self.p.land_size/2]) <= self.p.dist_per_tick[1]:
            
            # Move to the hive
            self.pos = [self.p.land_size/2,self.p.land_size/2]
            self.modinstance.beespace.move_to(self, self.pos)
            # Become inactive
            # Reset orientation
            self.orientation = model47.nprandom.uniform(low = 0.0, high = 360.0) 
            self.state = "inactive"
            
        # Otherwise, the bee moves towards the hive    
        else:
            
            # Differences between the bee's and the hive's x and y coordinates
#             x_dif = abs(current_x - self.p.land_size/2)
#             y_dif = abs(current_y - self.p.land_size/2)
            
            # Find the angle from the bee to the hive
            global_angle = np.arctan2(self.p.land_size/2-current_y, self.p.land_size/2 - current_x)

            # Correct for bees current orientation
            move_angle = global_angle - self.orientation 
            # DOUBLE-CHECK THIS
            
            # Move towards the hive at a predetermined distance per time tick
            self.move_radially(self, self.p.dist_per_tick[1], move_angle)
            self.state = "returning"


██████████████████████████████████████████

### Hive Class
The hive is a pretty boring kind of agent. It sits in the same spot the entire simulation run. I've made it an agent because I think recording the colony-level parameters as attributes of the hive is more intuitive than the abstract agentpy variable-recording syntax. For instance, later on we'll probably want to track the total amount of nectar the bees have collected, among other things. I think it would be useful, or at least intuitive, to group these values related to foraged nectar by "giving" them to the hive agent.

Although I'm not really using its physical location for much at the moment (I think in my bee calculations, I've hard-coded its position to be the center of the square map), having it present gives us flexibility later on (ex. checking if it's within a certain radius using `select` in agentpy).

In [89]:
#%%bgc #fff5b5

class Hive(ap.Agent):
    """ 
    The hive, located at the center of the map
    """
    
    def setup(self):  
        """ Initializing Hive agent variables"""
        self.modinstance = model47
        
        # Place the hive at the center of the map
        self.pos = self.coords = (self.p.land_size/2,self.p.land_size/2)
        # Note that the map is square, with side length land_size
        # Right now, the map is being drawn in the first quadrant,
        # with the origin in the lower left corner.
        
        # Variables used in color-coding animation
        self.kind = "hive"
        self.state = "hive"
        self.time_searching = 666 # need to include bc hive is in beespace
        self.num_trips = 666
        
        # Eventually there will be more hive-level variables,
        # Such as total nectar collected, etc.
        # I think this means the hive will have to access info from bees,
        # so I might need to define 
        # self.Model = model47
        # within this class as well
        
    def print_debug(self):
#         nearby_bees = self.modinstance.beespace.select(self.pos, 5).to_list()
        searchlist = []
#         triplist = []
#         for b in nearby_bees:
#             #searchlist.append(b.id)
#             searchlist.append(b.time_searching)
#             triplist.append(b.num_trips)
#         print("TIME SEARCHING:")
#         print(sorted(searchlist))
#         print("NUMBER OF TRIPS:")
#         print(triplist)


██████████████████████████████████████████

### Resource Class

In [69]:
#%%bgc #E2F0CB

class Resource(ap.Agent):
    """ 
    A patch of flowering plants, represented by a circle
    """
    
    def setup(self):  
        """ 
        Initializing variables belonging to Resource agents 
        """
        self.modinstance = model47
        
        #Attributes
        self.radius = self.p.radius # Currently, all patches have the same radius
        self.trips = model47.nprandom.integers(low = self.p.trips_range[0], high = self.p.trips_range[1]) 
        
        # Set parameters used for color-coding in animation
        self.kind = "resource"
        self.state = "resource"
        
        # Gravity model parameters
        self.mass = model47.nprandom.uniform(low = 1, high = 5) 
        self.velocity = [0,0] #[UNITS: spatial unit per time step]
        
    def setup_pos(self, space):
        """
        Set up relationship with continuous space
        """
        # Note that right now, the random placement of the plants 
        # happens in the ForagingModel class
        
        self.space = space 
        self.neighbors = space.neighbors 
        self.pos = space.positions[self]
        # Define the range of (x,y) values that fall in a particular resource's bounding box 
        self.range_x = (self.pos[0] - self.radius, self.pos[0] + self.radius)
        self.range_y = (self.pos[1] - self.radius, self.pos[1] + self.radius)
        # DO THESE VARIABLES ALSO AHVE TO BE INITIALIZED IN SETUP,
        # OR IS IT OK BC THEY'RE IN SETUP_POS?
        
    def feel_gravity(self):
        """
        Update positions based on the "gravitation" of surrounding plants
        """
        
        contributions_xy = []
        
        for r in self.model.resources:
            
            # Calculate distance to each of the other plants
            dist_x = self.model.plantspace.positions[r][0] - self.pos[0]
            dist_y = self.model.plantspace.positions[r][1] - self.pos[1]
            
            #Multiple ways to get large effects
            # One is to keep regular G and multiply masses by a factor
            # Another is just to adjust G
            # I think it's easier to adjust G, as we're not going for physical accuracy
            
            G = 6.674 * 10 **(-6)
            m1 = self.mass
            m2 = r.mass
            
            if dist_x == 0:
                Fx = 0
            else:
                xdir = dist_x / abs(dist_x)
                Fx = xdir * G * m1 * m2 / (dist_x)**2
            
            if dist_y == 0:
                Fy = 0
            else:
                ydir = dist_y / abs(dist_y)
                Fy = ydir * G * m1 * m2 / (dist_y)**2
            
            contributions_xy.append([Fx, Fy])
            
        contrib_x = np.array(contributions_xy)[:,0]
        contrib_x = contrib_x.tolist()
        contrib_y = np.array(contributions_xy)[:,1]
        contrib_y = contrib_y.tolist()
        

        sum_forces_x = sum(contrib_x)
        sum_forces_y = sum(contrib_y)
        accel_xy = [sum_forces_x/self.mass, sum_forces_y/self.mass] 
        # This is fine bc we assign nonzero mass
        
        self.velocity = [self.velocity[0]+ accel_xy[0], self.velocity[1]+ accel_xy[1]]
        
        self.pos += self.velocity
        self.modinstance.plantspace.move_to(self, self.pos)
        
        out_of_range = self.pos[0] >= self.p.land_size or self.pos[1] >= self.p.land_size or self.pos[0]<= 0 or self.pos[1] <= 0
        
        if out_of_range:
            self.pos[0] = model47.nprandom.uniform(low = 0.0, high = self.p.land_size)
            self.pos[1] = model47.nprandom.uniform(low = 0.0, high = self.p.land_size)
            self.velocity = [0,0]
        
        self.modinstance.plantspace.move_to(self, self.pos)
        #[:,1]
        
        # Use that to calculate attraction
        # Include a clause for excluding self
        # i.e. if dist >0, do calculation. else, contribution = 0
        
        

██████████████████████████████████████████

### Gravity model for patchy plant layout

Below is an attempt at clumpy landscape generation (given a desired R_value we want the environment to have).

**The cells necessary to run the GravityModel cell are:**

* Import cell (red, top of document)
* Resource class (green, directly above)
* Parameter cell (purple, bottom of document)

**Here are the different parameters that can impact the R_value (and other things) about the resources' spatial distribution.**

Parameter cell:
* `land_size`, the size of a side of our square environment
* `n_resources`, the number of resources appearing in that environment

GravityModel cell:
* Distribution of plant "masses"
* `G`, the gravitational constant

**Notes**
* Plants are allowed to overlap each other (regardless of their radius)
* Whenever a plant reaches or exceeds the landscape's boundary, it is placed back into the landscape and is given an initial velocity of 0.

**Things to try / change**
* Make the order in which the plants have their positions updated is randomized
    * Or, make it so all calculations are done based on positions in the previous timestep
    * Include a "next position" variable to alter, then change all positions at once
* Adjust distribution of masses- something with a heavy tail
* Add friction to allow the plants to slow down, see if it converges
* Eliminate overlaps (factoring in the individual plants' radii)
* Limit graviational pull to plants within a certain radius
* Display input parameters

In [70]:
#%%bgc #B5EAD7
class GravityModel(ap.Model):
    """ 
    Building the landscape
    """

    def setup(self):
        """ 
        Initialize the agents
        """
        
        # Create ground
        self.plantspace = ap.Space(self, shape=[self.p.land_size]*2, Torus = False)
        
        # Create flowering plants
        random_coord_array = np.random.uniform(0, self.p.land_size, size=(self.p.n_resources, 2))
        random_coord_list = random_coord_array.tolist()
        
        self.resources = ap.AgentList(self, self.p.n_resources, Resource)
        self.plantspace.add_agents(self.resources, random_coord_list, random=False) # Scatter them randomly
        self.resources.setup_pos(self.plantspace)
        
        # Create a list of all agents in the model
        self.agents = self.resources
    
        # Inital R value calculation
        self.calculate_r_val(random_coord_array)
        print("Starting R_val: ", self.R_val)
        

    #===========================================================#
    
    def update(self):  
        """ 
        Record variables after setup and each step. 
        """
        #TBH not entirely sure how to use this
        #self.record(self.R_val) #throws an error
    
    #===========================================================#
        
    def step(self):   
        """ 
        Define the models' events per simulation step (tick). 
        """
        # Adjust plant positions
        self.resources.feel_gravity()
        
        # Get updated positions
        pos_vals = self.plantspace.positions.values()
        pos_array = np.array([list(item) for item in pos_vals])
        self.calculate_r_val (pos_array)
        
    #===========================================================#
        
    def calculate_r_a(self, random_coord_array):
        """
        Pass in a 2d array, 
        """
        nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(random_coord_array)
        nn_distances_array, indices = nbrs.kneighbors(random_coord_array)
        stack = np.vstack(nn_distances_array)
        distances = stack[:,1]
        return (np.mean(distances))
    
    #===========================================================#
        
    def calculate_r_val (self, positions_array):
        r_a = self.calculate_r_a(positions_array)
        r_e = 0.5 * self.p.land_size * (1/np.sqrt(self.p.n_resources))
        self.R_val = r_a/r_e
        return(self.R_val)
        
        
#===========================================================#
#                     Generate Landscape                    #
#===========================================================#

model47 = GravityModel(parameters)
control = ips.Control(model47, parameters, variables = ('t'))

scatterplot = ips.Scatterplot(
    control,
    xy=lambda m: m.plantspace.positions.values(),
    c=lambda m: m.agents.state
)

lineplot = ips.Lineplot(control, ('R_val'))

AppLayout(left_sidebar=control,
          center=lineplot,#scatterplot,
          right_sidebar = scatterplot,
          pane_widths=['325px', 1, 1],
          height='300px')

AppLayout(children=(Control(layout=Layout(grid_area='left-sidebar'), parameters={'fps': 5, 'steps': 300, 'land…

Starting R_val:  0.9224936679605911


██████████████████████████████████████████

In [71]:
#%%bgc #C7CEEA

class ForagingModel(ap.Model):
    """ 
    Agent-based model that simulates a very 
    simplified version of bee foraging behavior 
    """

    def setup(self):
        """ 
        Initialize the agents
        """
        
        # Create ground
        self.beespace = ap.Space(self, shape=[self.p.land_size]*2, Torus = False)
        self.plantspace = ap.Space(self, shape=[self.p.land_size]*2, Torus = False)

        # Create flowering plants
        self.resources = ap.AgentList(self, self.p.n_resources, Resource)
        self.plantspace.add_agents(self.resources, random=True) # Scatter them randomly
        self.resources.setup_pos(self.plantspace)
        
        # Create hive
        self.hive= ap.AgentList(self, 1, Hive)
        self.beespace.add_agents(self.hive, [(self.p.land_size/2,self.p.land_size/2)], random=False)
        
        # Create bees
        self.bees = ap.AgentList(self, self.p.population, Bee)
        # Place them at the hive
        self.beespace.add_agents(self.bees, [[self.p.land_size/2,self.p.land_size/2]]*self.p.population , random=False) 
        self.bees.setup_pos(self.beespace)
        
        # Create a list of all agents in the model
        self.agents = self.resources +  self.hive + self.bees 
        
        # Initialize counts of bees in each state
        self.num_inactive = self.p.population
        self.num_searching = 0
        self.num_foraging = 0
        self.num_returning = 0
        
    def update(self):  
        """ 
        Record variables after setup and each step. 
        """
        
        for s in ["inactive", "searching", "foraging", "returning"]:
            n_agents = len(self.bees.select(self.bees.state == s))
            self[s] = n_agents / self.p.population 
            self.record(s)

    def step(self):   
        """ 
        Define the models' events per simulation step (tick). 
        """
        
        # Organize the bees into agentlists based on their states
        inactive_bees = self.bees.select(self.bees.state == "inactive")
        searching_bees = self.bees.select(self.bees.state == "searching")
        foraging_bees = self.bees.select(self.bees.state == "foraging")
        returning_bees = self.bees.select(self.bees.state == "returning")
        
        # Count the number of bees in each state
        self.num_inactive = len(inactive_bees)
        self.num_searching = len(searching_bees)
        self.num_foraging = len(foraging_bees)
        self.num_returning = len(returning_bees)

        # Have the bees do things based on their states
        inactive_bees.be_inactive()
        searching_bees.search_new()
        foraging_bees.forage()
        returning_bees.go_home()
        
        # DEBUG CLUMPING AROUND HIVE
        self.hive.print_debug()

    def end(self):     
        """ Record evaluation measures at the end of the simulation. """
        # Record final evaluation measures
        self.report('Max foragers', max(self.log['foraging'])) #haha self-report

██████████████████████████████████████████

### Parameters

Below is where we define the parameters for the model. Within the model *and* agent classes, you can ask for a parameter value (ex. `steps`) by calling `self.p.steps`. Below is a description of the different parameters

Category       | Name              | Units          | Description
:-------------:| :-------------:   | :-------------:| :-------------
**General**    |                   |                | 
               |     `fps`         |  frames/second | Range of speeds for animation slider
               |     `steps`       | time ticks     | Number of time ticks to run the simulation for
**Landscape**  |                   |                | 
               |    `land_size`    |   meters       | Size of one side of the square landscape
               |    `n_resources`  |  # of plants   | Number of resources on environment
               |        `radius`   |   meters       | The radius of each plant (same for all)
               |`trips_range`      | # trips        | The number of trips of nectar foraging a resource can support
 **Bee Setup** |                   |                | 
               |   `population`    |   # of bees    | The number of bees in the colony who can forage
               | `dist_per_tick`   |   meters/tick  | The speed a bee flies at when returning to the hive
               |`ticks_on_resource`|    time ticks  | The amount of time a bee spends foraging at a specific resource before returning to the hive               
               |`give_up_threshold`| time ticks     | The amount of time a bee spends unsuccessfully searching before giving up and going home
**Bee Probabilities**|             |                | 
               |`p_random_forage`  |     unitless   | The probability an inactive bee will start foraging
               | `p_abandon_search`|     unitless   | The probability a searching bee will give up and go home
                              
I still have to decide how I want to define the units for space and time ticks within the model. As a default, I've said that spatial units are in meters, and have kept everything in terms of time ticks. This has just been to get the model up and running.

The conversion between time ticks and hours/minutes/seconds will impact the following parameters:
* `steps`: Need to decide what a reasonable real-world duration we want to show is
* `dist_per_tick`: Based on bee foraging velocity literature
* `ticks_on_resource`: Based on bee movement within a patch/ foraging duration literature
* `give_up_threshold`: Based on bee foraging behavior literature

#### Notes on specific parameters / potential changes

* The units for `land_size`, `radius`, and `dist_per_tick` might change. They all need to be coordinated, i.e. the units for one cannot be modified without adjusting the units for the others (or introducing a conversion). I have set the default to be meters, but we might find that we need to use some sort of scale factor in order to render the size of environment we're interested in.


* Right now, the value of the `dist_per_tick` parameter is not based on the literature- I just wanted to get something up and running. It's also worth noting that when searching, the bee's distance per time tick is chosen from a uniform distribution from `(0, dist_per_tick)`, and when the bee returns to the hive, it moves at `dist_per_tick`. I'm not sure how realistic this is, but it can be easily modified. I'm not sure whether it'd be helpful to include a separate parameter for defining the search movement as well?


* The `radius` parameter will probably be changed from an integer to a list of variables defining a distribution. (I'm not sure yet what kind of distribution that would be.)


* The `p_random_forage` and `p_abandon_search` parameters are currently constants. I need to look into the literature, but I think it would make sense for them to be functions of other parameters. For instance, `p_random_forage` might depend on the total number of inactive bees, or the amount of nectar collected. Similarly, I can see `p_abandon_search` depending on distance from the hive and time spent searching. I'm not sure how much detail we want to add into the model (and can't remember what Sam Woodman used for these values), but I just wanted to flag that these parameters could be replaced with lists of coefficients to pass into a probability-calculation function.


* Right now, `p_abandon_search` isn't being used: instead, `give_up_threshold` is providing a max time that bees spend searching before they give up and go home. I think we'll want to be using both of theme parameters (i.e. a bee can give up before reaching the max `give_up_threshold`, but it can't keep searching indefinitely).

In [123]:
#%%bgc #c9a2ca

parameters = { 
    # General Setup
    'fps': ap.IntRange(1, 20, 5),
    'steps':300,
    # Environment Setup
    'land_size': 50,
    'n_resources': 100,
    'radius': 0.5,
    'trips_range': (1,2),
    # Bee Setup
    'population':100,
    'dist_per_tick': (1,3), #range of distances a bee travels per tick, max is used for return
    'ticks_on_resource': 30, #time spent foraging on resource
    'give_up_threshold': 50, #threshold for time outside before giving up
    # Bee Probabilities
    'p_random_forage': 0.5,
    'p_abandon_search': 0.05,
}


██████████████████████████████████████████

## Running the model

This is the cell that actually runs the model. It is supposed to generate an interactive visualization with a control panel on the left, a line graph in the center, and an animation of agent movements on the right.

Right now, the only parameter that can be altered through the control panel is `fps`, the frames-per-second of the animation. Changing it is helpful for slowing down and speeding up the visualization.

The center graph represents the number of bee agents in each state: inactive, searching, foraging, and returning. 

*Note: Right now, this is **not** incorporating the landscape generated by the gravity model.*

### Known issues

Unfortunately, at the moment, the color-coding in the center graph do **not** correspond to the colors on the right.

You might notice some weird circles appearing towards the end of the animation- that's a result of a bug with the equation I'm using to calculate return angle (it's on my to-do list!).

Finally, there is a *big* problem with the color-coding in the animation on the right. The *idea* is for each agent to be color-coded according to its `state_number`, an integer representing either the agent's identity or current status. The states are as follows:

| `agent.state`   | Agent         | Description
| :-------------: | :-------------:| :-------------:
Inactive          | 🐝| At the hive
Searching         | 🐝| Randomly looking for flowers
Foraging          | 🐝| Gathering nectar from a plant
Returning         | 🐝| Going back to hive
Resource          | 🌱| "I'm a plant!"
 Hive              | 🗄| "I'm a hive!"



In [124]:
#%%bgc #ecbdd0

model47 = ForagingModel(parameters)
control = ips.Control(model47, parameters, variables = ('t', 'num_searching'))

def both_space_positions(m):
    i2 = m.beespace.positions.values()
    #print(i1)
    i1 = m.plantspace.positions.values()
    return itertools.chain(i1, i2)

scatterplot = ips.Scatterplot(
    control,
    xy=lambda m: both_space_positions(model47), #m.space.positions.values(),
    c=lambda m: m.agents.state
)


lineplot = ips.Lineplot(control, ('num_inactive', 'num_searching', 'num_foraging', 'num_returning'))

AppLayout(left_sidebar=control,
          center=lineplot,
          right_sidebar = scatterplot,
          pane_widths=['325px', 1, 1],
          height='300px')

AppLayout(children=(Control(layout=Layout(grid_area='left-sidebar'), parameters={'fps': 5, 'steps': 300, 'land…

██████████████████████████████████████████

██████████████████████████████████████████

██████████████████████████████████████████

## Stackplot
Below is code to generate a stackplot of the different bee states. Because the number of bees is constant, the stackplot provides a nice way to see how the relative proportions of different states changes over time. However, I've found that I prefer the line graphs to visualize the amount of bees in each state, because it's far easier to see how the number of bees in a particular state changes- you just have to look at one line, instead of how one block of color changes relative to the other three blocks. 

In [14]:
#%%bgc #dec1ff

def foraging_stackplot(data, ax):
    """ Stackplot of people's condition over time. """
    x = data.index.get_level_values('t')
    y = [data[var] for var in ['inactive', 'searching', 'foraging', 'returning']]

    sns.set()  # Set seaborn theme for colors & lines
    ax.stackplot(x, y, 
                 labels=['Inactive', 'Searching', 'Foraging', 'Returning'],
                 colors = ['r', 'b', 'g', 'y'])    

    ax.legend()
    ax.set_xlim(0, max(1, len(x)-1))
    ax.set_ylim(0, 1)
    ax.set_xlabel("Time steps")
    ax.set_ylabel("Percentage of population")
    plt.show()

██████████████████████████████████████████

██████████████████████████████████████████

In [10]:
#Note to self on array-list conversion syntax
cat = np.array([[1,2],[3,4],[5,6],[7,8]])
print(cat[:,1])

[2 4 6 8]


██████████████████████████████████████████